In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels

/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/jy_anbe_py38/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/jy_anbe_py38/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
from bean.mapping._supporting_fn import _get_allele_from_alignment
from bean import Allele,Edit

In [3]:
edit_dict = {"A":"G", "T":"C"}
revcomp_dict = {"A":"T", "T":"A", "C":"G", "G":"C"}

In [4]:
var_info_tbl = pd.read_excel("../../resources/LDLvar/20221013_LDLvar_simpleZscores_credset.xlsx", index_col=0, header=1)
amplicon_info = pd.read_csv("../../resources/atac_seq/030123_ATACseq_info.csv", ).set_index("gRNA")

In [5]:
var_ids = amplicon_info.index.tolist()
#var_ids.remove("rs116734477_Maj_ABE_51")

In [6]:
fraction_dict = {0.5:(1,1), 1:(0,2), 0:(2,0), 0.333: (2,1), 0.667:(1,2)}
def get_variant_info(var_id):
    var_alleles = var_info_tbl.loc[var_id, ["A1", "A2", "HepG2 alternate allele fraction"]]
    print(var_alleles)
    if var_id == "rs771555783_Maj_ABE_309":
        ref_bases = ["A"]
#     elif var_id == "rs116734477_Maj_ABE_51":
#         return None
    else:
        n_A1, n_A2 = fraction_dict[var_alleles["HepG2 alternate allele fraction"]]
        ref_bases = []
        if n_A1 > 0 and var_alleles.A1:
            ref_bases.append(var_alleles.A1)
        if n_A2 > 0 and var_alleles.A2:
            ref_bases.append(var_alleles.A2)
        assert len(ref_bases) > 0, (var_id, var_alleles)
    #var_alleles['edited'] = var_alleles[var_id].map(lambda b: edit_dict[b] if b in edit_dict.keys() else np.nan)
    #var_alleles = var_alleles.loc[~var_alleles.edited.isnull()]
    var_chrom = int(var_info_tbl.loc[var_id, "CHR"])
    var_pos = int(var_info_tbl.loc[var_id, 'HG19 BP'])
    var_alleles.columns=["allele", "ref", "edited"]
    
    amplicon_start = amplicon_info.loc[var_id, "Varpos_fw"]-4
    amplicon_seq = amplicon_info.loc[var_id, "Amplicon_fw (reference)"].replace("N", "")
    amplicon_base = amplicon_seq[amplicon_start]
        
    return var_alleles, var_chrom, var_pos, ref_bases, amplicon_start, amplicon_base, amplicon_seq

In [7]:
var_info_dict = {}
for var_id in var_ids:
    print(var_id)
    vdf, vc, vp,var_refs, varpos_amplicon,  amplicon_base, amplicon_seq = get_variant_info(var_id)
    vdict = {}
    vdict["chrom"] = vc
    vdict["pos"] = vp
    vdict["amplicon_pos"] = varpos_amplicon
    vdict["refs"] = var_refs
    vdict["amplicon_base"] = amplicon_base
    vdict["amplicon_seq"] = amplicon_seq
    vdict["vdf"] = vdf
    var_info_dict[var_id] = vdict

rs4149309_Maj_ABE_206
A1                                   A
A2                                   T
HepG2 alternate allele fraction    0.5
Name: rs4149309_Maj_ABE_206, dtype: object
rs10107182_Min_ABE_351
A1                                   C
A2                                   T
HepG2 alternate allele fraction    1.0
Name: rs10107182_Min_ABE_351, dtype: object
rs3767844_Maj_ABE_192
A1                                   A
A2                                   G
HepG2 alternate allele fraction    0.5
Name: rs3767844_Maj_ABE_192, dtype: object
rs62084210_Maj_ABE_250
A1                                     A
A2                                     G
HepG2 alternate allele fraction    0.333
Name: rs62084210_Maj_ABE_250, dtype: object
rs115421711_Maj_ABE_45
A1                                   A
A2                                   G
HepG2 alternate allele fraction    0.0
Name: rs115421711_Maj_ABE_45, dtype: object
rs1434282_Min_ABE_412
A1                                   C
A2               

In [8]:
def _get_allele_from_alignment(
    ref_aligned: str,
    query_aligned: str,
    offset: int,
    strand: int,
    start_pos: int,
    end_pos: int,
    positionwise_quality: np.ndarray = None,
    quality_thres: float = -1,
):
    # Include N, no quality filter
    assert len(ref_aligned) == len(query_aligned)
    allele = Allele()
    ref_gaps = 0
    alt_gaps = 0
    alt_seq_len = len(query_aligned) - query_aligned.count("-")
    if positionwise_quality is None:
        # alt_position_is_good_quality = np.ones(alt_seq_len, dtype=bool)
        alt_position_is_good_quality = np.array(
            [c != "N" for c in query_aligned.replace("-", "")]
        )
    else:
        assert len(positionwise_quality) == alt_seq_len
        alt_position_is_good_quality = positionwise_quality > quality_thres
    for i in range(len(ref_aligned)):
        if ref_aligned[i] == query_aligned[i]:
            continue
        ref_base = ref_aligned[i]
        alt_base = query_aligned[i]
        if alt_base != "-":
            alt_base_is_good_quality = alt_position_is_good_quality[i - alt_gaps]
        else:
            alt_base_is_good_quality = True
        if ref_base == "-":
            ref_gaps += 1
        elif alt_base == "-":
            alt_gaps += 1
        ref_pos = i - ref_gaps
        allele.add(
            Edit(
                rel_pos=ref_pos,
                ref_base=ref_base,
                alt_base=alt_base,
                offset=offset,
                strand=strand,
            )
        )
    return allele

In [9]:
def get_edit(row, target_pos):
    return _get_allele_from_alignment(row.Reference_Sequence, row.Aligned_Sequence, offset=0, strand=1, start_pos=target_pos-20, end_pos=target_pos+20)

In [10]:
def get_snp_pos(var_id):
    var_file = "/PHShome/jr1025/projects/ANBE/data/HepG2_reference/ENCFF713BPG.vcf.gz"
    vdf, vc, vp, varpos_amplicon, amplicon_base, amplicon_seq = get_variant_info(var_id)
    pos = f"{vc}:{vp-varpos_amplicon + 20}-{vp+len(amplicon_seq)-varpos_amplicon-20}"
    command = f"bcftools query -r {pos} -f '%POS\t%REF\t%ALT'\n {var_file}"
    proc = subprocess.Popen(("module load bcftools; "+command).split(" "), stdout=subprocess.PIPE, shell=True)
    (out, err) = proc.communicate()
    return out, err

def get_pos_edit_rate(aftbl):
    total_counts = aftbl['#Reads'].sum()
    aftbl = aftbl.loc[aftbl.be_allele != "",:].copy()
    aftbl['edits'] = aftbl.be_allele.map(lambda a: a.split(","))
    edit_tbl = aftbl.explode("edits")
    edit_tbl['pos'] = edit_tbl.edits.map(lambda e: int(e.split(":")[0]))
    per_pos_edits = edit_tbl.groupby("pos")["#Reads"].sum()
    per_pos_edit_rates = per_pos_edits / aftbl['#Reads'].sum()
    return np.where(per_pos_edit_rates > 0.5)[0]
    
def format_allele_freq_table(path, varpos_amplicon):
    aftbl = pd.read_csv(path, sep='\t')
    aftbl["allele"] = aftbl.apply(lambda row:get_edit(row, varpos_amplicon),axis=1)
    aftbl['be_allele'] = aftbl.allele.map(filter_out_indels)
    alt_positions = get_pos_edit_rate(aftbl)
    aftbl['err'] = aftbl.be_allele.map(lambda a:filter_allele_by_pos(a, alt_positions, varpos_amplicon)[0])
    aftbl['N'] = aftbl.be_allele.map(lambda a:filter_allele_by_pos(a, alt_positions, varpos_amplicon)[1])
    aftbl['target_proximal_N'] = aftbl.be_allele.map(lambda a:filter_allele_by_pos(a, alt_positions, varpos_amplicon)[2])
    aftbl['pos_allele'] = aftbl.be_allele.map(lambda a:filter_allele_by_pos(a, alt_positions, varpos_amplicon)[3])
    aftbl['clean'] = aftbl.err <= 1
    return aftbl

def get_edit_counts(aftbl, target_pos, amplicon_base):
    ref, alt = zip(*aftbl.pos_allele.map(lambda a: get_pos_base(a, target_pos, amplicon_base)))
    aftbl['ref'] = ref
    aftbl['alt'] = alt
    return aftbl

In [11]:
def filter_allele_by_pos(allele, list_pos, target_pos):
    # filters allele by variant included in list_pos or near target_pos
    edits = []
    n_out_edits = 0
    n_lq = 0
    n_crit_lq = 0
    for edit in allele.split(","):
        if edit == "": continue
        alt = edit.split(">")[-1]
        pos = int(edit.split(":")[0])
        if alt == "N": 
            n_lq += 1
            if abs(pos-target_pos) < 5:
                n_crit_lq += 1
        elif abs(pos-target_pos) < 5:
            edits.append(edit)
        else:
            n_out_edits += 1
    return n_out_edits, n_lq, n_crit_lq, ",".join(sorted(edits))

def get_pos_base(allele, target_pos, amplicon_base):
    edits = []
    for edit in allele.split(","):
        if edit == "": continue
        pos = int(edit.split(":")[0])
        if pos == target_pos:
            ref, alt = edit.split(":")[-1].split(">")
            return ref, alt
    return amplicon_base, amplicon_base

def filter_out_indels(allele):
    edits = []
    for edit in allele.edits:
        if edit.ref_base != "-" and edit.alt_base != "-":
            edits.append(str(edit))
    return ",".join(sorted(edits))

In [12]:
counts_list = []

for i, var_id in enumerate(var_ids):
    print(var_id)
    varpos_amplicon = var_info_dict[var_id]["amplicon_pos"]
    amplicon_base = var_info_dict[var_id]["amplicon_base"]
    for rep in [1,2,3,"C"]:
        for cond in ["Ser", "SS"]:
            for exp in ["ATAC_Seq", "GDNA"]:
                try:
                    nt_freq_path = f"../../results/atac_seq/crispresso_runs_indiv_demuxed/CRISPResso_on_{rep}-{cond}_20-locus_{exp}.{var_id}/{var_id[:21]}.Nucleotide_frequency_table.txt"
                    nt_freq_tbl = pd.read_csv(nt_freq_path, sep='\t', header=None, index_col = 0)
                except FileNotFoundError:
                    print(f"{nt_freq_path} not found")
                    continue
                
                refs = nt_freq_tbl.iloc[0,:].tolist()
                nt_freq_tbl = nt_freq_tbl.iloc[1:,:]
                target_pos_edits = nt_freq_tbl.iloc[:,[varpos_amplicon]].reset_index()
                assert refs[varpos_amplicon] == amplicon_base
                target_pos_edits.columns=["base", "count"]
                target_pos_edits["amplicon_ref"] = refs[varpos_amplicon]
                target_pos_edits["var_id"] = var_id
                target_pos_edits["rep"] = rep
                target_pos_edits["cond"] = cond
                target_pos_edits["exp"] = exp
                counts_list.append(target_pos_edits)


rs4149309_Maj_ABE_206
rs10107182_Min_ABE_351
rs3767844_Maj_ABE_192
rs62084210_Maj_ABE_250
rs115421711_Maj_ABE_45
rs1434282_Min_ABE_412
rs4719853_Maj_ABE_214
rs771555783_Maj_ABE_309
../../results/atac_seq/crispresso_runs_indiv_demuxed/CRISPResso_on_1-Ser_20-locus_GDNA.rs771555783_Maj_ABE_309/rs771555783_Maj_ABE_3.Nucleotide_frequency_table.txt not found
../../results/atac_seq/crispresso_runs_indiv_demuxed/CRISPResso_on_2-Ser_20-locus_GDNA.rs771555783_Maj_ABE_309/rs771555783_Maj_ABE_3.Nucleotide_frequency_table.txt not found
../../results/atac_seq/crispresso_runs_indiv_demuxed/CRISPResso_on_3-Ser_20-locus_GDNA.rs771555783_Maj_ABE_309/rs771555783_Maj_ABE_3.Nucleotide_frequency_table.txt not found
../../results/atac_seq/crispresso_runs_indiv_demuxed/CRISPResso_on_3-SS_20-locus_GDNA.rs771555783_Maj_ABE_309/rs771555783_Maj_ABE_3.Nucleotide_frequency_table.txt not found
../../results/atac_seq/crispresso_runs_indiv_demuxed/CRISPResso_on_C-Ser_20-locus_GDNA.rs771555783_Maj_ABE_309/rs771555783_M

In [13]:
def plot_error_stats(aftbl, axs, label="", title=""):
    sns.scatterplot(data=aftbl.groupby("err")["#Reads"].sum().reset_index(), x='err', y='#Reads', ax = axs[0], s=10)
    axs[0].set_title("Mismatch")
    axs[0].set_ylabel(f"{title}\n#Reads")
    sns.scatterplot(data=aftbl.groupby("N")["#Reads"].sum().reset_index(), x='N', y='#Reads', ax = axs[1], s= 10)
    axs[1].set_title("# Low quality nt")
    sns.scatterplot(data=aftbl.groupby("target_proximal_N")["#Reads"].sum().reset_index(), x='target_proximal_N', y='#Reads', ax = axs[2], label=label, s = 10)
    axs[2].set_title("# Low quality nt near target")
    axs[2].get_legend().remove()
    

In [14]:
counts_df = pd.concat(counts_list)

In [15]:
def check_valid(ref, alt, var_id):
    '''Checks if ref>alt base is valid for var_id. 
    Returns 0 for ref, 1 for alt, and -1 for invalid'''
    var_info = var_info_dict[var_id]
    assert ref ==  var_info["amplicon_base"]
    if alt == "N":
        return -1
    if ref not in var_info["refs"]:
        # ex) 
        assert len(var_info["refs"]) == 1
        ref_base = var_info["refs"][0]
        if alt == ref_base: return 0
        if alt == edit_dict[ref_base]: return 1
        return -1
    possible_alts = []
    for r in var_info['refs']:
        if r in edit_dict.keys():
            if r == ref: continue
            possible_alts.append(edit_dict[r])
            
    if ref in edit_dict.keys():
        if edit_dict[ref] in var_info["refs"]:
            # Ambiguous edited vs other phase's WT
            return 0.5
        if alt in possible_alts:
            # Ambiguous WT vs other phase's edited
            return 0.5
        if edit_dict[ref] == alt: return 1
        if ref == alt: return 0
    phases = var_info["vdf"][:2].tolist()
    phases.remove(ref)
    if ref == alt: return 0
    if alt == phases[0]: return 0
    if phases[0] in edit_dict.keys() and alt == edit_dict[phases[0]]: return 1
    else:
        print(ref, alt, var_id)
        return -1

In [16]:
counts_df["valid_mt"] = counts_df.apply(lambda row: check_valid(row.amplicon_ref, row.base, row.var_id), axis=1)

A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A - rs4149309_Maj_ABE_206
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_Maj_ABE_45
A T rs115421711_Maj_ABE_45
A - rs115421711_Maj_ABE_45
A C rs115421711_M

In [17]:
var_counts_dict = {}
failed_variants = ["rs3767844_Maj_ABE_192", "rs62084210_Maj_ABE_250", "rs116734477_Maj_ABE_51", "rs35081008_Min_ABE_465", "rs4390169_Maj_ABE_208"] + ["rs771555783_Maj_ABE_309"] # needs phasing, not enough reads
pdfs = []
edit_assignments = []
for var_id in var_ids:
    pdf = pd.pivot(counts_df.loc[counts_df.var_id == var_id], index=["var_id", "exp", "cond", "rep",], columns=["amplicon_ref", "base",], values='count').astype(float)
    if len(pdf) == 0:
        failed_variants.append(var_id)
        print(f"ERROR: {var_id} does not have reads.")
    edit_assignment = counts_df.loc[counts_df.var_id == var_id, ["amplicon_ref", "base", "valid_mt"]].drop_duplicates()
    if len(edit_assignment.valid_mt.unique()) == 1:
        print(f"WARNING:{var_id} cannot be resolved.")
        if var_id not in failed_variants:
            print(f"ERROR: {var_id} not expected to fail but is not resolved.")
        failed_variants.append(var_id)
    pdf_t = pdf.T
    pdf_t['valid_mt'] = edit_assignment.set_index(["amplicon_ref", "base"])
    pdf_t = pdf_t.groupby("valid_mt").sum()
    pdf = pdf_t.T
    pdf.columns = pdf.columns.map({0:"ref", 0.5:"ambiguous", 1:"alt"})
    var_counts_dict[var_id] = (pdf, edit_assignment)
    pdf['status'] = {True:"FAILED", False:"SUCCESS"}[len(edit_assignment.valid_mt.unique()) == 1]
    pdfs.append(pdf)
    edit_assignment['label'] = edit_assignment.valid_mt.map({0:"ref", 0.5:"ambiguous", 1:"alt"})
    edit_assignment["var_id"] = var_id
    edit_assignment["status"] = {True:"FAILED", False:"SUCCESS"}[len(edit_assignment.valid_mt.unique()) == 1]
    edit_assignments.append(edit_assignment[["var_id", "amplicon_ref", "base", "label", "status"]])

In [18]:
resolved_counts = pd.concat(pdfs).reset_index()
edit_assignment_all = pd.concat(edit_assignments).drop_duplicates()

In [19]:
enrichment_results = []
ps = []
bs = []
ci_high = []
ci_low = []
interaction_ps = []
interaction_bs = []
interaction_ci_high = []
interaction_ci_low = []
for var_id in var_ids:
    if var_id in failed_variants: 
        ps.append(np.nan)
        bs.append(np.nan)
        ci_high.append(np.nan)
        ci_low.append(np.nan)
        interaction_ps.append(np.nan)
        interaction_bs.append(np.nan)
        interaction_ci_high.append(np.nan)
        interaction_ci_low.append(np.nan)
        continue
    print(var_id)
    pdf = resolved_counts.loc[resolved_counts.var_id == var_id].copy()
    pdf['sample'] = pdf["rep"].astype(str) + "_" + pdf["cond"]
    pdf["is_atac"] = pdf.exp == "ATAC_Seq"
    pdf["is_trt"] = pdf.rep != "C"
    #pdf = pdf.loc[pdf.rep != "C"]
    y, X = dmatrices('alt + ref ~ rep + cond + is_atac + cond*is_atac', data = pdf, return_type='dataframe')
    try:
        glm_binom = sm.GLM(y, X, family=sm.families.Binomial())
        glm_res = glm_binom.fit()
        print(glm_res.summary())
    except:
        print(pdf)
        ps.append(np.nan)
        bs.append(np.nan)
        ci_high.append(np.nan)
        ci_low.append(np.nan)
        interaction_ps.append(np.nan)
        interaction_bs.append(np.nan)
        interaction_ci_high.append(np.nan)
        interaction_ci_low.append(np.nan)
        continue
    if glm_res.pvalues["is_atac[T.True]"] < 0.05: 
        print(f"***Significant differential enrichment of ALT allele in ATAC-seq compared to GDNA: dlogit={glm_res.params['is_atac[T.True]']}***")
    df = pd.DataFrame(glm_res.summary().tables[1])
    df.columns=df.iloc[0,:]
    df = df.iloc[1:,:]
    df.insert(0,"variant", var_id)
    df.columns=["variant", "coef label", "coef", "std err", "z", "P>|z|", "[0.025", "0.975]"]
    enrichment_results.append(df)
    ps.append(glm_res.pvalues["is_atac[T.True]"])
    bs.append(glm_res.params['is_atac[T.True]'])
    ci_high.append(df.loc[df["coef label"].astype(str) == "is_atac[T.True]", "0.975]"].item())
    ci_low.append(df.loc[df["coef label"].astype(str) == "is_atac[T.True]", "[0.025"].item())
    interaction_ps.append(glm_res.pvalues["cond[T.Ser]:is_atac[T.True]"])
    interaction_bs.append(glm_res.params['cond[T.Ser]:is_atac[T.True]'])
    interaction_ci_high.append(df.loc[df["coef label"].astype(str) == "cond[T.Ser]:is_atac[T.True]", "0.975]"].item())
    interaction_ci_low.append(df.loc[df["coef label"].astype(str) == "cond[T.Ser]:is_atac[T.True]", "[0.025"].item())

rs4149309_Maj_ABE_206
                 Generalized Linear Model Regression Results                  
Dep. Variable:         ['alt', 'ref']   No. Observations:                   16
Model:                            GLM   Df Residuals:                        9
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -11.082
Date:                Tue, 22 Aug 2023   Deviance:                       5.5180
Time:                        15:38:37   Pearson chi2:                     5.66
No. Iterations:                    27                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------

                 Generalized Linear Model Regression Results                  
Dep. Variable:         ['alt', 'ref']   No. Observations:                   16
Model:                            GLM   Df Residuals:                        9
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -23.771
Date:                Tue, 22 Aug 2023   Deviance:                       7.0675
Time:                        15:38:37   Pearson chi2:                     6.22
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [20]:
enrichment_res_df = pd.concat(enrichment_results)
significance_df = pd.DataFrame({
    "var_id":var_ids, 
     "p|atac":ps, 
     "b|atac":bs, 
     "ci_high|atac":ci_high,
     "ci_low|atac":ci_low,
     "p|ser*atac":interaction_ps, 
     "b|ser*atac":interaction_bs,
     "ci_high|ser*atac":interaction_ci_high,
     "ci_low|ser*atac":interaction_ci_low,
})

In [21]:
# Manually fill in the data for variants that needed phasing
tbl = pd.read_csv("rs35081008_Min_ABE_465_result.csv", index_col=0
                 )
significance_df.loc[significance_df.var_id == "rs35081008_Min_ABE_465", 
                    ['p|atac', 
                     'b|atac', 
                     'ci_high|atac', 
                     'ci_low|atac', 
                     'p|ser*atac', 
                     'b|ser*atac', 
                     'ci_high|ser*atac', 
                     'ci_low|ser*atac']
                   ] = tbl.loc["is_atac[T.True]", ["P>|z|", "coef", "0.975]", "[0.025"]].tolist() + tbl.loc["cond[T.Ser]:is_atac[T.True]", ["P>|z|", "coef", "0.975]", "[0.025"]].tolist() 

In [22]:
for covariate in ['atac', 'ser*atac']:
    multicorrect_res = statsmodels.stats.multitest.multipletests(significance_df[f"p|{covariate}"].dropna(), method="fdr_bh", alpha=0.1)
    multicorrect_res = statsmodels.stats.multitest.multipletests(significance_df[f"p|{covariate}"].dropna(), method="fdr_bh", alpha=0.1)
    significance_df.loc[~significance_df[f"p|{covariate}"].isnull(), f"p_adj|{covariate}"] = multicorrect_res[1]
    significance_df.loc[~significance_df[f"p|{covariate}"].isnull(), f"sig(a=0.1)|{covariate}"] = multicorrect_res[0]

In [23]:
enrichment_results = []
ps = []
bs = []
for var_id in var_ids:
    if var_id in failed_variants: 
        ps.append(np.nan)
        bs.append(np.nan)
        interaction_ps.append(np.nan)
        interaction_bs.append(np.nan)
        continue
    print(var_id)
    pdf = resolved_counts.loc[resolved_counts.var_id == var_id].copy()
    pdf['sample'] = pdf["rep"].astype(str) + "_" + pdf["cond"]
    pdf["is_atac"] = pdf.exp == "ATAC_Seq"
    pdf["is_trt"] = pdf.rep != "C"
    #pdf = pdf.loc[pdf.rep != "C"]
    y, X = dmatrices('alt + ref ~ is_trt', data = pdf, return_type='dataframe')
    try:
        glm_binom = sm.GLM(y, X, family=sm.families.Binomial())
        glm_res = glm_binom.fit()
        print(glm_res.summary())
    except:
        print(pdf)
        ps.append(np.nan)
        bs.append(np.nan)
        interaction_ps.append(np.nan)
        interaction_bs.append(np.nan)
        continue
    if glm_res.pvalues["is_trt[T.True]"] >= 0.05: 
        print(f"***Editing not enriched in treatment sample***")
    df = pd.DataFrame(glm_res.summary().tables[1])
    df.columns=df.iloc[0,:]
    df = df.iloc[1:,:]
    df.insert(0,"variant", var_id)
    df.columns=["variant", "coef label", "coef", "std err", "z", "P>|z|", "[0.025", "0.975]"]
    enrichment_results.append(df)
    ps.append(glm_res.pvalues["is_trt[T.True]"])
    bs.append(glm_res.params['is_trt[T.True]'])

rs4149309_Maj_ABE_206
                 Generalized Linear Model Regression Results                  
Dep. Variable:         ['alt', 'ref']   No. Observations:                   16
Model:                            GLM   Df Residuals:                       14
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -24.519
Date:                Tue, 22 Aug 2023   Deviance:                       32.392
Time:                        15:38:38   Pearson chi2:                     39.3
No. Iterations:                    27                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -28.5

                 Generalized Linear Model Regression Results                  
Dep. Variable:         ['alt', 'ref']   No. Observations:                   16
Model:                            GLM   Df Residuals:                       14
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -969.59
Date:                Tue, 22 Aug 2023   Deviance:                       1867.9
Time:                        15:38:38   Pearson chi2:                 1.85e+03
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -8.9726      0.316    -28.

In [24]:
enrichment_res_df = pd.concat(enrichment_results)
ctrl_significance_df = pd.DataFrame({
    "var_id":var_ids, 
     "p|trt":ps, 
     "b|trt":bs, })

In [25]:
tbl = pd.read_csv("rs35081008_Min_ABE_465_ctrl_result.csv")
ctrl_significance_df.loc[ctrl_significance_df.var_id == "rs35081008_Min_ABE_465", ["p|trt", "b|trt"]] = tbl.loc[1, ["P>|z|", "coef"]].tolist()

In [26]:
for covariate in ['trt']:
    multicorrect_res = statsmodels.stats.multitest.multipletests(ctrl_significance_df[f"p|{covariate}"].dropna(), method="fdr_bh", alpha=0.1)
    ctrl_significance_df.loc[~ctrl_significance_df[f"p|{covariate}"].isnull(), f"p_adj|{covariate}"] = multicorrect_res[1]
    ctrl_significance_df.loc[~ctrl_significance_df[f"p|{covariate}"].isnull(), f"sig(a=0.1)|{covariate}"] = multicorrect_res[0]

## For unphasable unresolvable variants, ignore editing and calculate caQTL

In [27]:
unphasable_variants = ["rs3767844_Maj_ABE_192", "rs4390169_Maj_ABE_208"]

rs116734477_Maj_ABE_51: C/C, not edited.

In [28]:
heterozygous_variants = [var_id for var_id in var_ids if len(var_info_dict[var_id]['refs']) == 2]

In [29]:
unphasable_counts_list = []
bs = []
ps = []
pdfs = {}
ci_high = []
ci_low = []
interaction_ps = []
interaction_bs = []
interaction_ci_high = []
interaction_ci_low = []
enrichment_results = []
for var_id in heterozygous_variants:
    print(var_id)
    ref_base = var_info_dict[var_id]['amplicon_base']
    alt_base = [b for b in var_info_dict[var_id]['refs'] if b != ref_base][0]
    var_df = counts_df.loc[(counts_df.var_id == var_id) & (counts_df.base.isin([ref_base, alt_base]))]
    pdf = pd.pivot_table(var_df, index=["var_id", "exp", "cond", 'rep'], columns=["base",], values='count').astype(float).reset_index().rename(columns={ref_base:'amplicon_ref', alt_base:'HepG2_alt'})
    unphasable_counts_list.append(pdf)
    pdf['sample'] = pdf["rep"].astype(str) + "_" + pdf["cond"]
    pdf["is_atac"] = pdf.exp == "ATAC_Seq"
    pdf["is_trt"] = pdf.rep != "C"
    pdfs[var_id] = pdf
    y, X = dmatrices('amplicon_ref + HepG2_alt ~ rep + cond + is_atac + cond*is_atac', data = pdf, return_type='dataframe')
    try:
        glm_binom = sm.GLM(y, X, family=sm.families.Binomial())
        glm_res = glm_binom.fit()
        print(glm_res.summary())
    except:
        print(pdf)
        ps.append(np.nan)
        bs.append(np.nan)
        ci_high.append(np.nan)
        ci_low.append(np.nan)
        interaction_ps.append(np.nan)
        interaction_bs.append(np.nan)
        interaction_ci_high.append(np.nan)
        interaction_ci_low.append(np.nan)
        continue
    if glm_res.pvalues["is_atac[T.True]"] < 0.05: 
        print(f"***Significant differential enrichment of ALT allele in ATAC-seq compared to GDNA: dlogit={glm_res.params['is_atac[T.True]']}***")
    df = pd.DataFrame(glm_res.summary().tables[1])
    df.columns=df.iloc[0,:]
    df = df.iloc[1:,:]
    df.insert(0,"variant", var_id)
    df.columns=["variant", "coef label", "coef", "std err", "z", "P>|z|", "[0.025", "0.975]"]
    enrichment_results.append(df)
    ps.append(glm_res.pvalues["is_atac[T.True]"])
    bs.append(glm_res.params['is_atac[T.True]'])
    ci_high.append(df.loc[df["coef label"].astype(str) == "is_atac[T.True]", "0.975]"].item())
    ci_low.append(df.loc[df["coef label"].astype(str) == "is_atac[T.True]", "[0.025"].item())
    interaction_ps.append(glm_res.pvalues["cond[T.Ser]:is_atac[T.True]"])
    interaction_bs.append(glm_res.params['cond[T.Ser]:is_atac[T.True]'])
    interaction_ci_high.append(df.loc[df["coef label"].astype(str) == "cond[T.Ser]:is_atac[T.True]", "0.975]"].item())
    interaction_ci_low.append(df.loc[df["coef label"].astype(str) == "cond[T.Ser]:is_atac[T.True]", "[0.025"].item())

rs4149309_Maj_ABE_206
                       Generalized Linear Model Regression Results                       
Dep. Variable:     ['amplicon_ref', 'HepG2_alt']   No. Observations:                   16
Model:                                       GLM   Df Residuals:                        9
Model Family:                           Binomial   Df Model:                            6
Link Function:                             logit   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -68.196
Date:                           Tue, 22 Aug 2023   Deviance:                       36.224
Time:                                   15:38:38   Pearson chi2:                     36.6
No. Iterations:                                5                                         
Covariance Type:                       nonrobust                                         
                                  coef    std err          z      P>|z|      [

                       Generalized Linear Model Regression Results                       
Dep. Variable:     ['amplicon_ref', 'HepG2_alt']   No. Observations:                   16
Model:                                       GLM   Df Residuals:                        9
Model Family:                           Binomial   Df Model:                            6
Link Function:                             logit   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -729.34
Date:                           Tue, 22 Aug 2023   Deviance:                       1255.9
Time:                                   15:38:38   Pearson chi2:                 1.26e+03
No. Iterations:                                4                                         
Covariance Type:                       nonrobust                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
----

In [30]:
counts_df

,base,count,amplicon_ref,var_id,rep,cond,exp,valid_mt
0,A,305.0,A,rs4149309_Maj_ABE_206,1,Ser,ATAC_Seq,0.0
1,C,0.0,A,rs4149309_Maj_ABE_206,1,Ser,ATAC_Seq,0.5
2,G,1.0,A,rs4149309_Maj_ABE_206,1,Ser,ATAC_Seq,1.0
3,T,159.0,A,rs4149309_Maj_ABE_206,1,Ser,ATAC_Seq,0.0
4,N,7.0,A,rs4149309_Maj_ABE_206,1,Ser,ATAC_Seq,-1.0
...,...,...,...,...,...,...,...,...
1,C,8.0,T,rs1250259_Min_ABE_392,C,SS,GDNA,1.0
2,G,26.0,T,rs1250259_Min_ABE_392,C,SS,GDNA,0.5
3,T,76219.0,T,rs1250259_Min_ABE_392,C,SS,GDNA,0.0
4,N,5986.0,T,rs1250259_Min_ABE_392,C,SS,GDNA,-1.0


In [31]:
df = pdfs["rs4149309_Maj_ABE_206"]
df['lfc'] = df['HepG2_alt'] / df['amplicon_ref']

In [32]:
allele_enrichment_res_df = pd.concat(enrichment_results)
caqtl_significance_df = pd.DataFrame(
    {"var_id":heterozygous_variants, 
     "p|atac":ps, 
     "b|atac":bs, 
     "ci_high|atac":ci_high,
     "ci_low|atac":ci_low,
     "p|ser*atac":interaction_ps, 
     "b|ser*atac":interaction_bs,
     "ci_high|ser*atac":interaction_ci_high,
     "ci_low|ser*atac":interaction_ci_low,
    })

In [33]:
# Manually fill in the data for variants that needed phasing
tbl = pd.read_csv("rs35081008_Min_ABE_465_caqtl_result.csv", index_col=0
                 )
caqtl_significance_df.loc[caqtl_significance_df.var_id == "rs35081008_Min_ABE_465", 
                    ['p|atac', 
                     'b|atac', 
                     'ci_high|atac', 
                     'ci_low|atac', 
                     'p|ser*atac', 
                     'b|ser*atac', 
                     'ci_high|ser*atac', 
                     'ci_low|ser*atac']
                   ] = tbl.loc["is_atac[T.True]", ["P>|z|", "coef", "0.975]", "[0.025"]].tolist() + tbl.loc["cond[T.Ser]:is_atac[T.True]", ["P>|z|", "coef", "0.975]", "[0.025"]].tolist() 

tbl = pd.read_csv("rs62084210_Maj_ABE_250_caqtl_result.csv", index_col=0
                 )
caqtl_significance_df.loc[caqtl_significance_df.var_id == "rs62084210_Maj_ABE_250", 
                    ['p|atac', 
                     'b|atac', 
                     'ci_high|atac', 
                     'ci_low|atac', 
                     'p|ser*atac', 
                     'b|ser*atac', 
                     'ci_high|ser*atac', 
                     'ci_low|ser*atac']
                   ] = tbl.loc["is_atac[T.True]", ["P>|z|", "coef", "0.975]", "[0.025"]].tolist() + tbl.loc["cond[T.Ser]:is_atac[T.True]", ["P>|z|", "coef", "0.975]", "[0.025"]].tolist() 

In [34]:
for covariate in ['atac', 'ser*atac']:
    multicorrect_res = statsmodels.stats.multitest.multipletests(caqtl_significance_df[f"p|{covariate}"].dropna(), method="fdr_bh", alpha=0.1)
    multicorrect_res = statsmodels.stats.multitest.multipletests(caqtl_significance_df[f"p|{covariate}"].dropna(), method="fdr_bh", alpha=0.1)
    caqtl_significance_df.loc[~caqtl_significance_df[f"p|{covariate}"].isnull(), f"p_adj|{covariate}"] = multicorrect_res[1]
    caqtl_significance_df.loc[~caqtl_significance_df[f"p|{covariate}"].isnull(), f"sig(a=0.1)|{covariate}"] = multicorrect_res[0]

In [35]:
caqtl_significance_df

,var_id,p|atac,b|atac,ci_high|atac,ci_low|atac,p|ser*atac,b|ser*atac,ci_high|ser*atac,ci_low|ser*atac,p_adj|atac,sig(a=0.1)|atac,p_adj|ser*atac,sig(a=0.1)|ser*atac
0,rs4149309_Maj_ABE_206,1.421221e-25,-0.799128,-0.649,-0.949,8.429579e-02,-0.189066,0.026,-0.404,3.789923e-25,True,1.348733e-01,False
1,rs3767844_Maj_ABE_192,7.358820e-01,-0.056502,0.272,-0.385,3.872154e-01,0.188869,0.617,-0.239,7.934388e-01,False,5.162872e-01,False
2,rs62084210_Maj_ABE_250,4.570000e-01,-0.062300,0.102,-0.226,7.660000e-01,0.033000,0.25,-0.184,7.312000e-01,False,7.660000e-01,False
3,rs2618566_Min_ABE_453,7.111619e-194,0.128867,0.137,0.120,2.081110e-10,0.039112,0.051,0.027,2.844647e-193,True,4.162221e-10,True
4,rs35081008_Min_ABE_465,0.000000e+00,1.555300,1.58,1.531,0.000000e+00,-0.149300,-0.117,-0.182,0.000000e+00,True,0.000000e+00,True
5,rs4390169_Maj_ABE_208,6.847839e-02,0.005717,0.012,-0.000,1.803900e-20,0.042893,0.052,0.034,1.369568e-01,False,4.810400e-20,True
6,rs402348_Maj_ABE_201,7.934388e-01,0.011121,0.094,-0.072,7.382978e-01,-0.021374,0.104,-0.147,7.934388e-01,False,7.660000e-01,False
7,rs1250259_Min_ABE_392,5.843289e-01,-0.001645,0.004,-0.008,3.505914e-36,-0.055544,-0.047,-0.064,7.791052e-01,False,1.402366e-35,True


In [36]:
unresolved_counts = pd.concat(unphasable_counts_list).iloc[:,:-2]

In [37]:
all_counts = pd.concat([resolved_counts,unresolved_counts])

In [38]:
phased_variants = ["rs35081008_Min_ABE_465", "rs62084210_Maj_ABE_250"]
nonedited_variants = ['rs116734477_Maj_ABE_51']
all_counts.loc[all_counts.var_id.isin(phased_variants), 'status'] = 'Ambiguous edited vs. minor reference, phased'
all_counts.loc[all_counts.var_id.isin(nonedited_variants), 'status'] = 'Not edited (HepG2 does not have A or T ref)'
all_counts.loc[all_counts.var_id.isin(unphasable_variants),'status'] = 'Ambiguous edited vs. minor reference, unphasable'

In [39]:
with pd.ExcelWriter('20230822_ATAC_demuxed_analysis_result.xlsx') as writer:  
    all_counts.to_excel(writer, sheet_name="counts")
    edit_assignment_all.to_excel(writer, sheet_name="edit assignment")
    enrichment_res_df.to_excel(writer, sheet_name="edit enrichment")
    significance_df.to_excel(writer, sheet_name="edit enrichment summary")
    allele_enrichment_res_df.to_excel(writer, sheet_name="allele enrichment")
    caqtl_significance_df.to_excel(writer, sheet_name="allele enrichment summary")
    ctrl_significance_df.to_excel(writer, sheet_name='ctrl')